In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

from data_generator import DataGenerator

import model as model_util

from tqdm import tqdm_notebook as tqdm

import matplotlib.pyplot as plt

from numpy import array
import numpy as np


import datetime
from datetime import datetime

from tensorflow.keras.layers import BatchNormalization, GlobalMaxPool1D, Bidirectional, Dense, Flatten, Conv2D, LeakyReLU, Dropout, LSTM, GRU, Input
from tensorflow.keras import Model, Sequential
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator, pad_sequences


from tensorflow.keras import datasets, layers, models

import tensorflow as tf

#%load_ext tensorboard
print(tf.__version__)

2.1.0


In [2]:
def reset_seed():
    seed_value= 0
    # 1. Set `PYTHONHASHSEED` environment variable at a fixed value
    import os
    os.environ['PYTHONHASHSEED']=str(seed_value)
    # 2. Set `python` built-in pseudo-random generator at a fixed value
    import random
    random.seed(seed_value)
    # 3. Set `numpy` pseudo-random generator at a fixed value
    import numpy as np
    np.random.seed(seed_value)
    # 4. Set the `tensorflow` pseudo-random generator at a fixed value
    import tensorflow as tf
    tf.random.set_seed(seed_value)
    

In [26]:
import random

path = "drive/My Drive/model/"
path = "./"

trainX = np.load(path + "train" + "X.npy")
trainY = np.load(path + "train" + "Y.npy")
print("Train")
print(trainX.shape)
print(trainY.shape)

valX = np.load(path + "val" + "X.npy")
valY = np.load(path + "val" + "Y.npy")
print("Validation")
print(valX.shape)
print(valY.shape)

def get_balanced_set(X, Y):
    positiveX = [X[0]]
    positiveY = [Y[0]]
    negativeX = [X[0]]
    negativeY = [Y[0]]

    for i in tqdm(range(1, len(X)-1)):
        x = X[i]
        y = Y[i]
        if (y == 1):
            positiveX.append(x)
            positiveY.append(y)
        else:
            negativeX.append(x)
            negativeY.append(y)
   
    positiveX = np.array(positiveX)
    positiveY = np.array(positiveY)
    negativeX = np.array(negativeX)
    negativeY = np.array(negativeY)
    
    negative_num = len(positiveY)
    start_index = random.randint(0,(len(negativeX) - negative_num))
    end_index = start_index + negative_num
    
    print(start_index)
    print(end_index)
    
    trainX = np.concatenate((positiveX, negativeX[start_index:end_index]), axis=0)
    trainY = np.concatenate((positiveY, negativeY[start_index:end_index]), axis=0)
    return trainX, trainY, positiveX, positiveY, negativeX, negativeY


#trainX, trainY, positiveX, positiveY, negativeX, negativeY = get_balanced_set(trainX, trainY)

#valX, valY, positiveX_, positiveY_, negativeX_, negativeY_ = get_balanced_set(valX, valY)


print("positiveX {0}".format(positiveX.shape))
print("negativeX {0}".format(negativeX.shape))

print(trainX.shape)
print(trainY.shape)
print(valX.shape)
print(valY.shape)

Train
(883497, 40)
(883497,)
Validation
(2658, 40)
(2658,)
positiveX (121739, 40)
negativeX (761758, 40)
(883497, 40)
(883497,)
(2658, 40)
(2658,)


In [28]:
# define generator
n_input = 80

class CustomGen(TimeseriesGenerator):
    def __getitem__(self, idx):
        x,y = super().__getitem__(idx)
        return x, x

def get_gen(set_x, set_y, shuffle=True, batch_size=512):
    return CustomGen(set_x, set_y, length=n_input, batch_size=batch_size, shuffle=shuffle)

train_generator = get_gen(trainX, trainY, shuffle=True)

val_generator = get_gen(negativeX_, negativeX_, shuffle=False)

# number of samples
print('Samples: %d' % len(train_generator))

x, y = train_generator[0]

x_, y_ = val_generator[0]

features = x.shape[2]

print(x.shape)
print(y.shape)

Samples: 1726
(512, 80, 40)
(512, 80, 40)


In [29]:
from numpy import array
import numpy as np

import datetime
from datetime import datetime

from tensorflow.keras.layers import RepeatVector, TimeDistributed, Dense, Flatten, Conv2D, LeakyReLU, Dropout, LSTM, GRU, Input

from tensorflow.keras import Model, Sequential, regularizers

from tensorflow.keras import datasets, layers, models

import tensorflow as tf

dim = 20

def add_deep_layers(input_layer, units = features):
    x = Dense(units, activation='elu', kernel_regularizer=regularizers.l2(0.0001))(input_layer)
    x = Dense(units, activation='elu', kernel_regularizer=regularizers.l2(0.0001))(x)
    x = Dense(units, activation='elu', kernel_regularizer=regularizers.l2(0.0001))(x)
    return x

def get_model(seqs, features):
    reset_seed()
    inputX = Input(shape=(seqs,features))
    x = Dense(features, activation='elu')(inputX)
    x = Bidirectional(LSTM(units=features, return_sequences=True, activation='relu'))(x)
    x = add_deep_layers(x, features)
    x = add_deep_layers(x, features)
    x = add_deep_layers(x, features)
    x = add_deep_layers(x, features)
    x = add_deep_layers(x, features)
    x = Dense(30, activation='elu')(x)
    x = Dense(30, activation='elu')(x)
    x = Dense(30, activation='elu')(x)
    x = Dense(30, activation='elu')(x)
    x = Bidirectional(LSTM(30, activation='elu', return_sequences=False))(x)
    x = Dense(30, activation='elu')(x)
    x = Dense(dim, activation='elu', name= "encoder")(x)
    
    encoder = x

    # Decoder
    x = RepeatVector(seqs, name= "dencoder")(encoder)
    x = Bidirectional(LSTM(dim, return_sequences=True, activation='elu'))(x)
    x = add_deep_layers(x, dim)
    x = add_deep_layers(x, (features+dim)/2)
    x = add_deep_layers(x, features)
    x = add_deep_layers(x, features)
    x = add_deep_layers(x, features)
    x = add_deep_layers(x, features)
    x = add_deep_layers(x, features)
    x = add_deep_layers(x, features)
    x = add_deep_layers(x, features)
    x = Dense(features, activation='elu')(x)
    x = TimeDistributed(Dense(features, activation='elu'))(x)
    x = Dense(features)(x)

    model = Model(inputs=[inputX], outputs=x)
    
    return model

    
def load_model(filepath = "drive/My Drive/model/stock.h5"):
    model_loaded = tf.keras.models.load_model(filepath)
    return model_loaded

In [30]:

model = get_model(n_input, features)
#model = load_model(filepath = "drive/My Drive/model/stock.h5")
#model.summary()

#!rm "encoder.h5"

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=True),
                  loss='mean_squared_logarithmic_error',
                  #loss='mean_squared_error',
                  #loss='mean_absolute_error',
                  #loss='squared_hinge'
                  metrics=['accuracy'])

In [ ]:

filepath_encoder = "encoder.h5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath_encoder, monitor='loss',
                             verbose=1,
                             save_best_only=True,
                             load_weights_on_restart=True)

callbacks_list = [checkpoint]

history = model.fit(
    #x, x,
    train_generator,
    steps_per_epoch=len(train_generator)-1, 
    epochs=10, 
    verbose=1, 
    callbacks=callbacks_list)


  ...
    to  
  ['...']
Train for 1725 steps
Epoch 1/10
 121/1725 [=>............................] - ETA: 13:15 - loss: 8.6154 - accuracy: 0.0336

In [ ]:
def show_loss(history): 
  plt.plot(history.history['loss'])
  #plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()

show_loss(history)

In [ ]:
def show_accuracy(history):
  plt.plot(history.history['accuracy'])
  #plt.plot(history.history['val_loss'])
  plt.title('accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()

show_accuracy(history)

In [20]:
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import ops
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.losses import mean_squared_logarithmic_error, mean_squared_error

def error(y_true, y_pred):
    return np.mean(mean_squared_logarithmic_error(y_true, y_pred))

def show_result(modell):
    pred = modell.predict(x_)
    score1 = error(pred,x_)
    print(f"Out of Sample Score (RMSE): {score1}")
    pred = modell.predict(x)
    score2 = error(pred,x)
    print(f"Insample Normal Score (RMSE): {score2}")




In [21]:
show_result(model)


Out of Sample Score (RMSE): 5.585374476245208
Insample Normal Score (RMSE): 6.573929674782894


In [22]:
posX_, posY_ = get_gen(positiveX_, positiveX_, shuffle=True, batch_size=len(positiveX_))[0]

In [23]:
x_attack = posX_
pred = model.predict(x_attack)
score3 = error(pred,x_attack)
print(f"Attack Underway Score (RMSE): {score3}")

Attack Underway Score (RMSE): 6.11366899507781


In [ ]:
test = x_attack[0]

In [ ]:
error(model.predict(np.array([test])), test)

In [ ]:
model_loaded = model
encoder = Model(inputs=model_loaded.input, outputs=model_loaded.get_layer('encoder').output)

In [ ]:
encoder.predict(np.array([x_attack[20]]))

In [ ]:
!nvidia-smi

In [ ]:
test

In [ ]:
model.predict(np.array([test]))

In [ ]:
positiveY_[1]